In [13]:
import os
from hashlib import md5
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_page(url, city, df_list):
    try:
        response = requests.get(url=url)
        response.raise_for_status()
        page_text = response.text
        html = BeautifulSoup(page_text, 'html.parser')  # 使用html.parser作为备用解析器
        li_list = html.find('div', class_='listbox').find('ul').find_all('li')

        for li in li_list:
            # 解析网页中的景点信息并存储到字典中
            name = li.find('a', class_='titlink').text
            strategy_sum = li.find('div', class_='strategy_sum').text
            comment_sum = li.find('div', class_='comment_sum').text
            cur_star = li.find('span', class_='cur_star')['style'].split(':')[1].replace('%', '')
            ranking_sum = li.find('span', class_='ranking_sum').text
            desbox = li.find('div', class_='desbox')
            link = li.find('a', class_='imglink')['href']
            img = li.find('a', class_='imglink').find('img')['src']

            # 将景点信息存储到字典中
            citydata = {
                'key': md5(name.encode('utf-8')).hexdigest(),
                '城市': city,
                '景点名称': name,
                '攻略数量': strategy_sum,
                '评论数量': comment_sum,
                '星级': cur_star,
                '排名': ranking_sum,
                '简介': desbox.text,
                '链接': link,
                '图片': img
            }
            df_list.append(citydata)
            print(citydata)
    except Exception as e:
        print(f"Error occurred while processing {url}: {e}")

if __name__ == '__main__':
    filename = 'data.csv'
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=['key', '城市', '景点名称', '攻略数量', '评论数量', '星级', '排名', '简介', '链接', '图片'])
        df.to_csv(filename, encoding='utf-8-sig', index=False)

    urls = [
        {'city': '厦门', 'url': 'https://travel.qunar.com/p-cs299782-xiamen-jingdian-1-{page}', 'page': 1, 'max_page': 10},
        {'city': '三亚', 'url': 'https://travel.qunar.com/p-cs300188-sanya-jingdian-1-{page}',
            'page': 1, 'max_page': 10},
        {'city': '北京', 'url': 'https://travel.qunar.com/p-cs299914-beijing-jingdian-1-{page}',
            'page': 1, 'max_page': 10},
        {'city': '上海', 'url': 'https://travel.qunar.com/p-cs299878-shanghai-jingdian-1-{page}',
            'page': 1, 'max_page': 10},
        {'city': '南京', 'url': 'https://travel.qunar.com/p-cs299861-nanjing-jingdian-1-{page}',
            'page': 1, 'max_page': 10},
        {'city': '杭州', 'url': 'https://travel.qunar.com/p-cs300195-hangzhou-jingdian-1-{page}',
            'page': 1, 'max_page': 10},
        {'city': '衢州', 'url': 'https://travel.qunar.com/p-cs300184-quzhou-jingdian-1-{page}',
            'page': 1, 'max_page': 10},
        {'city': '绍兴', 'url': 'https://travel.qunar.com/p-cs300181-shaoxing-jingdian-1-{page}',
            'page': 1, 'max_page': 10},
        {'city': '成都', 'url': 'https://travel.qunar.com/p-cs300085-chengdu-jingdian-1-{page}',
            'page': 1, 'max_page': 10},
        {'city': '大连', 'url': 'https://travel.qunar.com/p-cs300134-dalian-jingdian-1-{page}',
            'page': 1, 'max_page': 10},
    ]
    
    df_list = []
    for url in urls:
        for page in range(1, url['max_page'] + 1):
            get_page(url['url'].format(page=page), url['city'], df_list)
            print(f"[{url['city']}] 第{page}页/共{url['max_page']}页")

    df = pd.DataFrame(df_list)
    df.drop_duplicates(subset=['景点名称'], inplace=True)
    df.to_csv(filename, encoding='utf-8-sig', index=False, mode='a', header=False)

{'key': 'c0a644a0e685a7f94dc6b61b94747cef', '城市': '厦门', '景点名称': '曾厝垵Zeng Cuo An Village', '攻略数量': '1055', '评论数量': '5797', '星级': '88', '排名': '厦门景点排名第1', '简介': '厦门文创新地标，美食与文化的激情碰撞，去寻找文艺外表下充满生命力的古朴村落。', '链接': 'https://travel.qunar.com/p-oi5475476-cengcuoan', '图片': 'https://img1.qunarzz.com/travel/d4/1803/8f/ac8c01cef2a16ab5.jpg_r_480x360x95_38d2b1c4.jpg'}
{'key': 'ddcfafc3044e88726b7944ef7fb7bbc0', '城市': '厦门', '景点名称': '鼓浪屿Gulangyu Island', '攻略数量': '1451', '评论数量': '30593', '星级': '88', '排名': '厦门景点排名第1', '简介': '世界文化遗产，特色店铺众多，建筑漂亮，生活气息和文艺范儿并重。', '链接': 'https://travel.qunar.com/p-oi715901-gulangyu', '图片': 'https://img1.qunarzz.com/travel/d4/1806/35/cdb229270d50d4b5.jpg_r_480x360x95_5d7807c8.jpg'}
{'key': '543d75b28ea81c939d7bf7c68dc35f44', '城市': '厦门', '景点名称': '厦门园林植物园Xiamen Botanical Garden', '攻略数量': '136', '评论数量': '1901', '星级': '92', '排名': '厦门景点排名第12', '简介': '环境很好的植物园，有多种稀有植物景观，也适合周末爬山休闲。', '链接': 'https://travel.qunar.com/p-oi708112-xiamenyuanlinzhiwuyuan', '图片': 'https://tr-osdcp.qunarzz.com

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import jieba

# 加载数据
data = pd.read_csv('data.csv')
data = data.drop_duplicates(subset=['景点名称'])
# 处理空值和确保文本数据为字符串类型
data['简介'] = data['简介'].fillna('').astype(str)

# 分词并去除停用词
data_intro = data['简介'].apply(lambda x: ' '.join(jieba.lcut(x)))

# 向量化文本数据
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data_intro)

# 使用K-means进行聚类
num_clusters = 5  # 假设我们想分成5个类别
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

# 将聚类结果添加到数据中
data['类别'] = kmeans.labels_

# 查看每个类别的代表性景点
for i in range(num_clusters):
    print(f"类别 {i}:")
    print(data[data['类别'] == i][['景点名称', '简介']].head())
    print("\n")

# 将聚类结果保存到 CSV 文件
data.to_csv('data_clustered.csv', encoding='utf-8-sig', index=False)

# 示例：对新景点进行分类
new_scenic = "这里有美丽的湖泊和山脉"
new_scenic_cut = ' '.join(jieba.lcut(new_scenic))
new_scenic_tfidf = tfidf_vectorizer.transform([new_scenic_cut])
predicted_cluster = kmeans.predict(new_scenic_tfidf)
print("Predicted Cluster for new scenic spot:", predicted_cluster[0])


类别 0:
                                        景点名称  \
1                         鼓浪屿Gulangyu Island   
2             厦门园林植物园Xiamen Botanical Garden   
3  厦门科技馆Xiamen Science and Technology Museum   
4     中山路步行街Zhongshan Road Pedestrian Street   
5                            环岛路Huandao Road   

                                         简介  
1           世界文化遗产，特色店铺众多，建筑漂亮，生活气息和文艺范儿并重。  
2             环境很好的植物园，有多种稀有植物景观，也适合周末爬山休闲。  
3              可以带孩子来这里感受新科技带来的乐趣，开拓孩子们的视野。  
4  厦门地标式商业街，欣赏连排的骑楼式建筑，搜罗地道的闽南小吃，感受鹭岛的独特魅力。  
5                       享受海边骑行的快乐，边骑边看美丽风景。  


类别 1:
     景点名称                                      简介
87  文曾路花溪  文曾路的花溪，每年的春节以后盛开。盛开期间，吸引了众多的市民，前来赏花摄影。


类别 2:
                景点名称                     简介
7  椰风寨Yefeng Village  很长的免费沙滩，游客不多，可以远眺金门岛。


类别 3:
    景点名称                                                 简介
69  景州乐园  景州乐园以“山水有情、人间有爱”为主题,园里的布局典雅、浪漫,五彩缤纷的花园里，错落着形态各...


类别 4:
                     景点名称                                      简介
0  曾厝垵Zeng Cuo An Villa

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 加载景点数据
data = pd.read_csv('data.csv')

data = data.dropna(subset=['简介'])

# 假设有一个队伍的特征
team_features = {
    'destination': '厦门',
    'departure_location': '上海',
    'travel_mode': '飞机',
    'team_type': '家庭',
    'travel_time': '暑假',
    'travel_budget': 5000,
    'max_members': 4,
    'current_members': 3,
    'admin_id': 123
}

# 基于队伍特征选择相关景点 data-dataframe 的类型；team_features-字典类型
def recommend_destination(data, team_features):
    # 根据目的地过滤景点
    filtered_data = data[data['城市'] == team_features['destination']]
    
    # 如果需要，可以根据更多的队伍特征进行过滤
    
    # 基于景点简介计算TF-IDF向量表示
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=1)  # 调整min_df参数

    tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_data['简介'].values.astype('U'))
    
    # 计算队伍特征与每个景点的相似度
    team_description = ' '.join([str(v) for v in team_features.values()])
    team_tfidf = tfidf_vectorizer.transform([team_description])
    similarities = cosine_similarity(team_tfidf, tfidf_matrix).flatten()
    
    # 返回相似度最高的前几个景点
    top_indices = similarities.argsort()[-5:][::-1]
    recommended_destinations = filtered_data.iloc[top_indices]
    return recommended_destinations

# 调用推荐函数
recommended_destinations = recommend_destination(data, team_features)
print(recommended_destinations)


                                  key  城市  \
737  56e19f1c1f0d136a51e0f8898139bc86  厦门   
96   4e37b6bc4bc84f615657ccb67a6610b1  厦门   
94   e91e55b178ece86c142b9549d6216347  厦门   
93   2c2d72f1613279c49b36223791553456  厦门   
92   543d75b28ea81c939d7bf7c68dc35f44  厦门   

                                          景点名称  攻略数量  评论数量  星级         排名  \
737                             华新路Huaxin Road    35    84  94  厦门景点排名第38   
96                      南普陀寺South Putuo Temple   730  4726  92   厦门景点排名第5   
94      中山路步行街Zhongshan Road Pedestrian Street   594  3372  90   厦门景点排名第3   
93   厦门科技馆Xiamen Science and Technology Museum    34   758  92  厦门景点排名第14   
92              厦门园林植物园Xiamen Botanical Garden   136  1901  92  厦门景点排名第12   

                                           简介  \
737            隐藏着厦门半个世纪的历史，漫步每栋建筑，寻找背后的历史故事。   
96             全国闻名的佛教寺院，寺内的素斋和素饼都获过奖，还可登高望远。   
94   厦门地标式商业街，欣赏连排的骑楼式建筑，搜罗地道的闽南小吃，感受鹭岛的独特魅力。   
93               可以带孩子来这里感受新科技带来的乐趣，开拓孩子们的视野。   
92              环境很好的